# Crypto Altcoin Analysis
This workbook will walk through extracting data from Coinbase on all altcoins available with a USD pairing, then clean data and do some performance and correlation analysis.



In [50]:
# Imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import numpy as np
import pandas as pd
import hvplot.pandas
import os
from pathlib import Path
from dotenv import load_dotenv
import csv
import cbpro
import requests
import time


In [51]:
# Setup Public Client for Coinbase
public_client = cbpro.PublicClient()

In [52]:
# Pull a list of all currencies (Assets and pairings)
currency_response_list = public_client.get_currencies()

In [53]:
# Pull only the assets that are marked as crypto

# Create empty list of cryptos
crypto_list = []

# If the asset is labeled as a crypto add to crypto list
for asset in currency_response_list:
    if asset['details']['type'] == 'crypto':
        crypto_list.append(asset['id'])
crypto_list.sort()
print(crypto_list)
print(len(crypto_list))

['1INCH', 'AAVE', 'ACH', 'ADA', 'ALGO', 'AMP', 'ANKR', 'ATOM', 'BAL', 'BAND', 'BAT', 'BCH', 'BNT', 'BOND', 'BTC', 'CGLD', 'CHZ', 'CLV', 'COMP', 'CRV', 'CTSI', 'CVC', 'DAI', 'DASH', 'DNT', 'DOGE', 'DOT', 'ENJ', 'EOS', 'ETC', 'ETH', 'FARM', 'FET', 'FIL', 'FORTH', 'GNT', 'GRT', 'GTC', 'ICP', 'KEEP', 'KNC', 'LINK', 'LOOM', 'LPT', 'LRC', 'LTC', 'MANA', 'MASK', 'MATIC', 'MIR', 'MKR', 'MLN', 'NKN', 'NMR', 'NU', 'OGN', 'OMG', 'OXT', 'PAX', 'PLA', 'POLY', 'QNT', 'RAI', 'REN', 'REP', 'RLC', 'RLY', 'SHIB', 'SKL', 'SNX', 'SOL', 'STORJ', 'SUSHI', 'TRB', 'UMA', 'UNI', 'USDC', 'USDT', 'WBTC', 'XLM', 'XRP', 'XTZ', 'YFI', 'ZEC', 'ZRX']
85


In [54]:
# Get a list of all Coinbase Products
all_products_list = public_client.get_products()
print(len(all_products_list))
#all_products_list

246


In [55]:
# Find product ids that have a USD quote_currency and create a list

# Create Empty List for USD paired products
usd_product_id_list = []
for product in all_products_list:
    if product['quote_currency'] == 'USD':
        usd_product_id_list.append(product['id'])
usd_product_id_list.sort()
print(usd_product_id_list)

['1INCH-USD', 'AAVE-USD', 'ACH-USD', 'ADA-USD', 'ALGO-USD', 'AMP-USD', 'ANKR-USD', 'ATOM-USD', 'BAL-USD', 'BAND-USD', 'BAT-USD', 'BCH-USD', 'BNT-USD', 'BOND-USD', 'BTC-USD', 'CGLD-USD', 'CHZ-USD', 'CLV-USD', 'COMP-USD', 'CRV-USD', 'CTSI-USD', 'DAI-USD', 'DASH-USD', 'DOGE-USD', 'DOT-USD', 'ENJ-USD', 'EOS-USD', 'ETC-USD', 'ETH-USD', 'FARM-USD', 'FET-USD', 'FIL-USD', 'FORTH-USD', 'GRT-USD', 'GTC-USD', 'ICP-USD', 'KEEP-USD', 'KNC-USD', 'LINK-USD', 'LPT-USD', 'LRC-USD', 'LTC-USD', 'MANA-USD', 'MASK-USD', 'MATIC-USD', 'MIR-USD', 'MKR-USD', 'MLN-USD', 'NKN-USD', 'NMR-USD', 'NU-USD', 'OGN-USD', 'OMG-USD', 'OXT-USD', 'PAX-USD', 'PLA-USD', 'POLY-USD', 'QNT-USD', 'RAI-USD', 'REN-USD', 'REP-USD', 'RLC-USD', 'RLY-USD', 'SKL-USD', 'SNX-USD', 'SOL-USD', 'STORJ-USD', 'SUSHI-USD', 'TRB-USD', 'UMA-USD', 'UNI-USD', 'USDT-USD', 'WBTC-USD', 'XLM-USD', 'XTZ-USD', 'YFI-USD', 'ZEC-USD', 'ZRX-USD']


In [56]:
# Make calls to the Products endpoint for each product_Id and pull candlestick data for the last 300 weeks
# GET https://api.pro.coinbase.com/products/<product-id>/candles
# Reponses contain the following metrics in a list of lists.
# [0] time bucket start time
# [1] low lowest price during the bucket interval
# [2] high highest price during the bucket interval
# [3] open opening price (first trade) in the bucket interval
# [4] close closing price (last trade) in the bucket interval
# [5] volume volume of trading activity during the bucket interval
# columns1 = ['Date'] + usd_product_id_list

def pull_historical_data(start,end):
    temp_df = pd.DataFrame()
    for item in good_list:
        historical = pd.DataFrame(public_client.get_product_historic_rates(item, start=start, end=end, granularity=86400))
        historical.columns=["Date","Open","High","Low",item,"Volume"]
        historical['Date'] = pd.to_datetime(historical['Date'], unit='s')
        historical.set_index('Date', inplace=True)
        historical.sort_values(by='Date', ascending=True, inplace=True)
        historical = historical.drop(['Open', 'High', 'Low', 'Volume'], axis=1)
        temp_df = pd.concat([temp_df, historical], axis=1)
    # print(f"temp_df 1: {temp_df}")
    return temp_df


main_df = pd.DataFrame()

# Apparently only some of these product ID's have data going back to 2019 - these are those
good_list = ['ALGO-USD', 'BCH-USD', 'BTC-USD', 'DASH-USD', 'EOS-USD', 'ETC-USD', 'ETH-USD', 'LINK-USD', 'LTC-USD', 'REP-USD', 'XLM-USD', 'XTZ-USD', 'ZRX-USD']        
    

date_start1 = '2019-01-01T00:00:00-06:00'
date_end1 = '2019-10-19T00:00:00-05:00'
date_start2 = '2019-10-20T00:00:00-06:00'
date_end2 = '2020-05-19T00:00:00-05:00'
date_start3 = '2020-05-20T00:00:00-06:00'
date_end3 ='2020-10-19T00:00:00-05:00'
date_start4 = '2020-10-20T00:00:00-06:00'
date_end4 = '2021-05-19T00:00:00-05:00'
date_start5 = '2021-05-20T00:00:00-06:00'
date_end5 ='2021-08-04T00:00:00-05:00'

main_df = pull_historical_data(date_start1, date_end1)
# clean up null values
main_df = main_df.dropna()
temp_df = pull_historical_data(date_start2, date_end2)
main_df = pd.concat([main_df, temp_df], axis=0)
temp_df = pull_historical_data(date_start3, date_end3)
main_df = pd.concat([main_df, temp_df], axis=0)
temp_df = pull_historical_data(date_start4, date_end4)
main_df = pd.concat([main_df, temp_df], axis=0)
temp_df = pull_historical_data(date_start5, date_end5)
main_df = pd.concat([main_df, temp_df], axis=0)

main_df


,ALGO-USD,BCH-USD,BTC-USD,DASH-USD,EOS-USD,ETC-USD,ETH-USD,LINK-USD,LTC-USD,REP-USD,XLM-USD,XTZ-USD,ZRX-USD
Date,,,,,,,,,,,,,
2019-09-17,0.3190,318.18,10185.39,94.337,4.067,6.279,207.95,1.62707,74.13,10.61,0.063411,1.1104,0.174939
2019-09-18,0.3411,324.04,10155.26,97.646,4.081,6.388,210.27,1.80999,77.75,10.73,0.082876,1.0861,0.192772
2019-09-19,0.3222,323.80,10275.01,99.935,3.968,6.226,220.80,1.87800,76.82,10.71,0.080944,1.0891,0.228500
2019-09-20,0.3201,315.75,10168.85,96.650,4.014,6.160,218.06,1.82052,75.10,10.91,0.071933,1.0628,0.232041
2019-09-21,0.3157,311.60,9973.99,92.817,3.997,6.114,214.80,1.88000,73.18,10.75,0.069626,1.0725,0.247124
2019-09-22,0.3003,306.75,10026.80,90.764,3.812,5.990,211.10,1.81940,72.26,10.47,0.068589,1.0479,0.237211
2019-09-23,0.2674,292.06,9693.74,86.545,3.682,5.809,200.80,1.80177,66.47,10.53,0.063373,1.0530,0.208438
2019-09-24,0.1947,246.26,8530.01,72.083,2.774,4.964,166.26,1.72850,55.50,8.93,0.054151,0.9199,0.191351
2019-09-25,0.2050,227.90,8438.35,73.423,2.865,4.860,170.00,1.75001,57.53,8.62,0.057395,0.8526,0.213803
